In [8]:
S3_location = "s3://ai2-llm/pretraining-data/sources/stack-dedup/v0/attributes/file_stats"
URLS_LIST = "../create/stack_dedup_v1.1_urls.txt"

In [3]:
import pandas as pd
import os

In [6]:
path = os.path.join(S3_location, "python", "data_0000.tsv")
df = pd.read_csv(path, sep="\t")

In [19]:
lang_urls = {}
with open(URLS_LIST) as f:
    lines = f.readlines()
    for line in lines:
        url = "/".join(line.split("/")[-2:]).strip().replace(".parquet", ".tsv")
        lang = url.split("/")[0]
        if lang in lang_urls:
            lang_urls[lang].append(url)
        else:
            lang_urls[lang] = [url]

In [23]:
lang = "abap"

all_dfs = []
for url in lang_urls[lang]:
    path = os.path.join(S3_location, url)
    df = pd.read_csv(path, sep="\t")
    all_dfs.append(df)

df = pd.concat(all_dfs)

In [27]:
df.to_csv("merged_abap.tsv", sep="\t")

In [25]:
df["line_count"].sum()

2924937

In [30]:
import json

In [32]:
with open("lang_files.json", "w") as f:
    json.dump(lang_urls, f)

In [33]:
with open("lang_files.json") as f:
    lf = json.load(f)

### 29 April 2023

In [3]:
MERGED_STATS = "../analyze/merged"

In [6]:
import pandas as pd
import os

In [7]:
lang = "python"
path = os.path.join(MERGED_STATS, f"merged_{lang}_stats.tsv")


In [11]:
df = df.drop(columns=["Unnamed: 0"])

In [25]:
df

,id,max_line_length,avg_line_length,num_characters,line_count,num_tokens_whitespace,alnum_prop
0,900000adc880b3f11205aecc535df29d629a6d0c,135,29.861111,3225,108,275,0.572093
1,90000123c3c98721346f41595fd534d124b2f830,69,21.275862,617,29,54,0.555916
2,9000014700819c031686be68800c6f771efb2695,124,41.670902,32795,787,2175,0.551517
3,90000324dfde4b624d3ad0979d89a438d0443b5c,119,49.500000,5544,112,411,0.619228
4,9000053b61a61a1ac2159e9bc1406f2b350532b6,79,35.037736,1857,53,272,0.653743
...,...,...,...,...,...,...,...
15148599,7cfffbeef703aae9a0eddf585ae768c178b3cf66,76,31.772727,699,22,53,0.666667
15148600,7cfffe678d106e7dbd5cd14e6764d27402d0a789,145,40.875000,2289,56,194,0.587156
15148601,7cfffeb814bc4b9668233c97fe82267404f3cd4c,83,35.000000,4235,121,329,0.604014
15148602,7cffff37b2d5bf8236ee0aa7d33d0bf05bac5452,1992,51.475728,5302,103,516,0.562052


In [23]:
filtered_df = df[(df["max_line_length"] > 1000) | (df["avg_line_length"] > 100) | (df["alnum_prop"] < 0.25)]

In [24]:
len(df) - len(filtered_df)

15036358

In [26]:
df["num_tokens_whitespace"].sum() - filtered_df["num_tokens_whitespace"].sum()

6446572637

In [27]:
import re

In [28]:
# Regex to strip repeated copyright comment blocks
CPAT = re.compile("copyright", re.IGNORECASE)
PAT = re.compile("/\\*[^*]*\\*+(?:[^/*][^*]*\\*+)*/")


def clean_copyright_comments(content: str):
    r = PAT.search(content)
    if r:
        # found one, now see if it contains "copyright", if so strip it
        span = r.span()
        sub = content[span[0] : span[1]]
        if CPAT.search(sub):
            # cut it
            content = content[: span[0]] + content[span[1] :]

        return content

    lines = content.split("\n")
    skip = 0

    # Greedy replace any file that begins with comment block, most
    # are copyright headers
    for k in range(len(lines)):
        if lines[k].startswith("//") or lines[k].startswith("#") or lines[k].startswith("--") or not lines[k]:
            skip = skip + 1
        else:
            break

    if skip:
        # we skipped, consume it
        content = "\n".join(lines[skip:])

    return content

In [29]:
S3_location = "s3://ai2-llm/pretraining-data/sources/stack-dedup/v0/documents"

In [92]:
path = os.path.join(S3_location, "python", "data_0000.jsonl.gz")
df = pd.read_json(path, lines=True, compression="gzip")

In [93]:
df["new_text"] = df["text"].apply(clean_copyright_comments)

In [94]:
df.to_json?

In [33]:
df[df["text"] != df["new_text"]]

,id,lang,text,metadata,timestamp,source,added,new_text
4,9000053b61a61a1ac2159e9bc1406f2b350532b6,python,# This code is based on the SOM class library....,"{'size': 1857, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,from benchmark import Benchmark\n\n\nclass Per...
7,9000070b2550faa1d17f57c00a6b8e20ec634906,python,# Built in python libs\nimport os\nimport time...,"{'size': 8214, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,import os\nimport time\n\n# Additional libs\ni...
8,9000085392939d04983239353323c8e90fdb8c0e,python,#!/usr/bin/env python3\n# Copyright 2013 The F...,"{'size': 1212, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,import argparse\nfrom fontTools import ttLib\n...
9,900008ef07bdbed94ca34c73bf749d14b9bceef0,python,# coding=utf-8\nimport json\nimport pandas as ...,"{'size': 5011, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,import json\nimport pandas as pd\nimport reque...
13,90000b542dae43e88da510338901abd49ec24789,python,#!/usr/bin/env python\n# coding=utf-8\nimport ...,"{'size': 1086, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,import numpy as np\nimport sys\n\nyears = sys....
...,...,...,...,...,...,...,...,...
152091,d5a39141c5e781840061ae86fd63aa6ce228d136,python,# Licensed under a 3-clause BSD style license ...,"{'size': 217, 'ext': 'py', 'max_stars_repo_pat...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,"from __future__ import print_function, divisio..."
152093,d5a3945cc71448fdd3ffa595169892fcf3cb5488,python,# -*- coding: utf-8 -*-\n# Generated by Django...,"{'size': 735, 'ext': 'py', 'max_stars_repo_pat...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,from __future__ import unicode_literals\n\nimp...
152094,d5a395ac1be24ebd884c61e4889ff3d31535262f,python,# -*- coding: utf-8 -*-\nfrom os import remove...,"{'size': 6386, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,"from os import remove, environ\nimport logging..."
152098,d5a39d463bd373321c5c571d263bbda7325d4811,python,#!/usr/bin/env python3\n#\n# Copyright 2014 Si...,"{'size': 1980, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,"__author__ = ""Simone Campagna""\n\n__all__ = [\..."


In [34]:
df

,id,lang,text,metadata,timestamp,source,added,new_text
0,900000adc880b3f11205aecc535df29d629a6d0c,python,"""""""\nbenchmarking et plotting some results for...","{'size': 3225, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,"""""""\nbenchmarking et plotting some results for..."
1,90000123c3c98721346f41595fd534d124b2f830,python,temp =[]\nprinc=[]\nmaior=menor=0\n\nwhile Tru...,"{'size': 617, 'ext': 'py', 'max_stars_repo_pat...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,temp =[]\nprinc=[]\nmaior=menor=0\n\nwhile Tru...
2,9000014700819c031686be68800c6f771efb2695,python,"""""""\nThis module implements an API for interac...","{'size': 32795, 'ext': 'py', 'max_stars_repo_p...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,"""""""\nThis module implements an API for interac..."
3,90000324dfde4b624d3ad0979d89a438d0443b5c,python,from ._object_env import ObjectEnv\nfrom ._obj...,"{'size': 5544, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,from ._object_env import ObjectEnv\nfrom ._obj...
4,9000053b61a61a1ac2159e9bc1406f2b350532b6,python,# This code is based on the SOM class library....,"{'size': 1857, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,from benchmark import Benchmark\n\n\nclass Per...
...,...,...,...,...,...,...,...,...
152097,d5a39bddc227e2320abd0ac10ad3a6e499a3455a,python,from rest_framework import serializers\nfrom a...,"{'size': 221, 'ext': 'py', 'max_stars_repo_pat...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,from rest_framework import serializers\nfrom a...
152098,d5a39d463bd373321c5c571d263bbda7325d4811,python,#!/usr/bin/env python3\n#\n# Copyright 2014 Si...,"{'size': 1980, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,"__author__ = ""Simone Campagna""\n\n__all__ = [\..."
152099,d5a39e6052dec6a6ade93459410a765d7d0c49c9,python,# SPDX-License-Identifier: Apache-2.0.\nimport...,"{'size': 6355, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,import logging\nfrom contextlib import closing...
152100,d5a39eb2f736ab4cd0cd0395f70d0a284bb8c52f,python,from django.core.management.base import BaseCo...,"{'size': 3686, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245,from django.core.management.base import BaseCo...


In [36]:
!pip install detect-secrets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00


In [48]:
from detect_secrets import SecretsCollection
from detect_secrets.settings import default_settings
from detect_secrets.core.scan import PotentialSecret, get_plugins, _process_line_based_plugins
from typing import Generator

the_str = "code_str.yml"

def scan_code(code: str) -> Generator[PotentialSecret, None, None]:
    if not get_plugins():   # pragma: no cover
        log.error('No plugins to scan with!')
        return

    has_secret = False
    for lines in [code.splitlines()]:
        for secret in _process_line_based_plugins(
            lines=list(enumerate(lines, start=1)),
            filename=the_str,
        ):
            has_secret = True
            yield secret

        if has_secret:
            break

class MySecretsCollection(SecretsCollection):
    def scan_str(self, code_str: str):
        for secret in scan_code(code_str):
            self[the_str].add(secret)

def check_for_secrets(code: str):
    secrets = MySecretsCollection()
    with default_settings():
        secrets.scan_str(code)

    secrets_json = secrets.json()
    if secrets_json:
        for secret in secrets_json[the_str]:
            print("verified", secret["is_verified"])
        return True
    return False

In [57]:
check_for_secrets('hello = "T2w5Vl9ZTUIzVzE5dTlBblUtRlo6MHNBYWxhbHVTeGFodUpUOWIybkNNZw=="')

verified False


True

In [58]:
check_for_secrets('contact = akshitab@allenai.org')

False

In [50]:
check_for_secrets("print('hello')")

False

In [53]:
df["text"].head(500).apply(check_for_secrets)

verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False
verified False


0      False
1      False
2      False
3      False
4      False
       ...  
495    False
496    False
497    False
498    False
499    False
Name: text, Length: 500, dtype: bool

In [59]:
dsets = default_settings()

In [60]:
dsets

In [45]:
print(df.iloc[39]["text"])

"""
Django settings for mysite project.

Generated by 'django-admin startproject' using Django 3.2.

For more information on this file, see
https://docs.djangoproject.com/en/3.2/topics/settings/

For the full list of settings and their values, see
https://docs.djangoproject.com/en/3.2/ref/settings/
"""

from pathlib import Path

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(__file__).resolve().parent.parent


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/3.2/howto/deployment/checklist/

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = 'django-insecure-$su^!=k#jx24b_mg1(oi)j7vgio%9gtbb+)uv1mw420r#mp%ru'

# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = True

ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',

In [90]:
path = os.path.join(S3_location, "jsoniq", "data_0000.jsonl.gz")
df = pd.read_json(path, lines=True, compression="gzip")

In [91]:
df["text"].values[1]

'{\n  "tag_name": "v1.7.0",\n  "name": "The Aristotle release",\n  "draft": false,\n  "prerelease": true,\n  "body": $changelog\n}'

In [1]:
df

NameError: name 'df' is not defined

### 30 April 2023

In [2]:
import pandas as pd
import os

In [7]:
S3_location = "s3://ai2-llm/pretraining-data/sources/stack-dedup/v1/attributes"
path = os.path.join(S3_location, "python", "data_0000.tsv")
df = pd.read_csv(path, sep="\t")

In [11]:
df[df["id"].isna()]

,id,alnum_prop,line_count,num_tokens_whitespace,num_alpha,alpha_count,max_line_length,avg_line_length,alnum_count,num_characters
784,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1082,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1252,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1580,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1740,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
150483,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
151091,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
151473,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
151594,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0


In [15]:
df[df["num_tokens_whitespace"] < 50]

,id,alnum_prop,line_count,num_tokens_whitespace,num_alpha,alpha_count,max_line_length,avg_line_length,alnum_count,num_characters
5,9000066e02e8dd2095aa95d6ac7ca85ad4c9e016,0.800000,1.0,4.0,NaN,20.0,24.0,25.000000,20.0,25.0
8,9000085392939d04983239353323c8e90fdb8c0e,0.694737,17.0,46.0,NaN,330.0,61.0,27.941176,330.0,475.0
10,90000a1b36aa3c75157aac2116dd2271317d6683,0.763006,8.0,12.0,NaN,132.0,82.0,21.625000,132.0,173.0
18,900013ccc8655d877d6553a8120f47a07a5f8cba,0.468992,14.0,42.0,NaN,112.0,31.0,18.428571,121.0,258.0
22,90001690169c6ec7744d2037ad6d3c145ce3a539,0.725552,14.0,27.0,NaN,220.0,44.0,22.642857,230.0,317.0
...,...,...,...,...,...,...,...,...,...,...
152088,d5a38f0dacc713ae708435c74429561e89ad8b0b,0.622378,7.0,23.0,NaN,89.0,51.0,20.428571,89.0,143.0
152091,d5a39141c5e781840061ae86fd63aa6ce228d136,0.692810,7.0,18.0,NaN,106.0,47.0,21.857143,106.0,153.0
152093,d5a3945cc71448fdd3ffa595169892fcf3cb5488,0.590634,26.0,45.0,NaN,368.0,107.0,25.461538,391.0,662.0
152096,d5a39b1a95d588c6e471fa3b22cdd5d0c2d956c7,0.441860,25.0,47.0,NaN,182.0,36.0,17.200000,190.0,430.0


In [14]:
df

,id,alnum_prop,line_count,num_tokens_whitespace,num_alpha,alpha_count,max_line_length,avg_line_length,alnum_count,num_characters
0,900000adc880b3f11205aecc535df29d629a6d0c,0.572093,108.0,275.0,NaN,1759.0,135.0,29.861111,1845.0,3225.0
1,90000123c3c98721346f41595fd534d124b2f830,0.555916,29.0,54.0,NaN,333.0,69.0,21.275862,343.0,617.0
2,9000014700819c031686be68800c6f771efb2695,0.551517,787.0,2175.0,NaN,17876.0,124.0,41.670902,18087.0,32795.0
3,90000324dfde4b624d3ad0979d89a438d0443b5c,0.619228,112.0,411.0,NaN,3364.0,119.0,49.500000,3433.0,5544.0
4,9000053b61a61a1ac2159e9bc1406f2b350532b6,0.452482,32.0,74.0,NaN,299.0,39.0,22.031250,319.0,705.0
...,...,...,...,...,...,...,...,...,...,...
152097,d5a39bddc227e2320abd0ac10ad3a6e499a3455a,0.746606,9.0,18.0,NaN,165.0,62.0,24.555556,165.0,221.0
152098,d5a39d463bd373321c5c571d263bbda7325d4811,0.736344,49.0,146.0,NaN,1009.0,81.0,28.020408,1011.0,1373.0
152099,d5a39e6052dec6a6ade93459410a765d7d0c49c9,0.561273,188.0,465.0,NaN,3534.0,91.0,33.595745,3545.0,6316.0
152100,d5a39eb2f736ab4cd0cd0395f70d0a284bb8c52f,0.554259,95.0,225.0,NaN,2016.0,84.0,38.800000,2043.0,3686.0


In [16]:
S3_location = "s3://ai2-llm/pretraining-data/sources/stack-dedup/v1/documents"
path = os.path.join(S3_location, "python", "data_0000.jsonl.gz")
dfx = pd.read_json(path, lines=True, compression="gzip")

In [20]:
fdf = pd.merge(df, dfx, on="id")

In [64]:
def filter_by_filecontent_stats(instance_attributes) -> bool:

    if instance_attributes["max_line_length"] > 1000:
        return False

    if instance_attributes["avg_line_length"] > 100:
        return False

    if instance_attributes["alnum_prop"] < 0.25:
        return False

    if instance_attributes["num_tokens_whitespace"] < 50:
        return False

    if instance_attributes["alpha_count"] / instance_attributes["num_tokens_whitespace"] < 1.5:
        return False

    return True

In [65]:
fdf

,id,alnum_prop,line_count,num_tokens_whitespace,num_alpha,alpha_count,max_line_length,avg_line_length,alnum_count,num_characters,lang,text,metadata,timestamp,source,added
0,900000adc880b3f11205aecc535df29d629a6d0c,0.572093,108.0,275.0,NaN,1759.0,135.0,29.861111,1845.0,3225.0,python,"""""""\nbenchmarking et plotting some results for...","{'size': 3225, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
1,90000123c3c98721346f41595fd534d124b2f830,0.555916,29.0,54.0,NaN,333.0,69.0,21.275862,343.0,617.0,python,temp =[]\nprinc=[]\nmaior=menor=0\n\nwhile Tru...,"{'size': 617, 'ext': 'py', 'max_stars_repo_pat...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
2,9000014700819c031686be68800c6f771efb2695,0.551517,787.0,2175.0,NaN,17876.0,124.0,41.670902,18087.0,32795.0,python,"""""""\nThis module implements an API for interac...","{'size': 32795, 'ext': 'py', 'max_stars_repo_p...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
3,90000324dfde4b624d3ad0979d89a438d0443b5c,0.619228,112.0,411.0,NaN,3364.0,119.0,49.500000,3433.0,5544.0,python,from ._object_env import ObjectEnv\nfrom ._obj...,"{'size': 5544, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
4,9000053b61a61a1ac2159e9bc1406f2b350532b6,0.452482,32.0,74.0,NaN,299.0,39.0,22.031250,319.0,705.0,python,from benchmark import Benchmark\n\n\nclass Per...,"{'size': 1857, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151695,d5a39bddc227e2320abd0ac10ad3a6e499a3455a,0.746606,9.0,18.0,NaN,165.0,62.0,24.555556,165.0,221.0,python,from rest_framework import serializers\nfrom a...,"{'size': 221, 'ext': 'py', 'max_stars_repo_pat...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
151696,d5a39d463bd373321c5c571d263bbda7325d4811,0.736344,49.0,146.0,NaN,1009.0,81.0,28.020408,1011.0,1373.0,python,"__author__ = ""Simone Campagna""\n\n__all__ = [\...","{'size': 1980, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
151697,d5a39e6052dec6a6ade93459410a765d7d0c49c9,0.561273,188.0,465.0,NaN,3534.0,91.0,33.595745,3545.0,6316.0,python,import logging\nfrom contextlib import closing...,"{'size': 6355, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
151698,d5a39eb2f736ab4cd0cd0395f70d0a284bb8c52f,0.554259,95.0,225.0,NaN,2016.0,84.0,38.800000,2043.0,3686.0,python,from django.core.management.base import BaseCo...,"{'size': 3686, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245


In [72]:
fdf = fdf[fdf.apply(filter_by_filecontent_stats, axis=1)]

In [67]:
len(fdf), len(df)

(151700, 152102)

In [69]:
len(dfx)

152102

In [70]:
df.columns

Index(['id', 'alnum_prop', 'line_count', 'num_tokens_whitespace', 'num_alpha',
       'alpha_count', 'max_line_length', 'avg_line_length', 'alnum_count',
       'num_characters'],
      dtype='object')

In [74]:
fdf[dfx.columns]

,id,lang,text,metadata,timestamp,source,added
0,900000adc880b3f11205aecc535df29d629a6d0c,python,"""""""\nbenchmarking et plotting some results for...","{'size': 3225, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
1,90000123c3c98721346f41595fd534d124b2f830,python,temp =[]\nprinc=[]\nmaior=menor=0\n\nwhile Tru...,"{'size': 617, 'ext': 'py', 'max_stars_repo_pat...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
2,9000014700819c031686be68800c6f771efb2695,python,"""""""\nThis module implements an API for interac...","{'size': 32795, 'ext': 'py', 'max_stars_repo_p...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
3,90000324dfde4b624d3ad0979d89a438d0443b5c,python,from ._object_env import ObjectEnv\nfrom ._obj...,"{'size': 5544, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
4,9000053b61a61a1ac2159e9bc1406f2b350532b6,python,from benchmark import Benchmark\n\n\nclass Per...,"{'size': 1857, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
...,...,...,...,...,...,...,...
151693,d5a397f162ac2d68e3358e46d19d308a8c88ab7e,python,import torch\nimport gym\nimport numpy as np\n...,"{'size': 2915, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
151696,d5a39d463bd373321c5c571d263bbda7325d4811,python,"__author__ = ""Simone Campagna""\n\n__all__ = [\...","{'size': 1980, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
151697,d5a39e6052dec6a6ade93459410a765d7d0c49c9,python,import logging\nfrom contextlib import closing...,"{'size': 6355, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
151698,d5a39eb2f736ab4cd0cd0395f70d0a284bb8c52f,python,from django.core.management.base import BaseCo...,"{'size': 3686, 'ext': 'py', 'max_stars_repo_pa...",2022-12-01,stack-dedup,2023-03-17T09:18:30.577245
